In [1]:
import gym
import numpy as np
import pandas as pd
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 
from empyrical import sharpe_ratio, sortino_ratio
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
import talib
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
import itertools
import warnings
import optuna
import torch

In [2]:
def resample_data(data):
    df=data.copy()
    # Assuming your DataFrame is named 'df' and it's a cuDF DataFrame
    df['time'] = pd.to_datetime(df['time'])
    df = df.set_index('time')

    # Resample and aggregate for each interval
    for interval in ['5T', '15T', '30T']:
        resampled = df.resample(interval).agg({'Open': 'first','High': 'max','Low': 'min','Close': 'last','Volume': 'sum'})
        # Create new column names based on the interval
        label = interval.replace('T', 'min')
        df[f'Open_{label}'] = resampled['Open']
        df[f'High_{label}'] = resampled['High']
        df[f'Low_{label}'] = resampled['Low']
        df[f'Close_{label}'] = resampled['Close']
        df[f'Volume_{label}'] = resampled['Volume']

   # Since resampling might introduce NaNs for periods without data, you might need to handle these
   # For example, you can fill NaNs with the last valid observation using fillna(method='ffill')
    df = df.fillna(method='ffill')

    # Reset the index if needed
    df = df.reset_index()
    return df

In [3]:
def calculate_amihud_illiquidity(close, volume):
    """
    Calculate Amihud Illiquidity

    Parameters:
    close (pd.Series): Series of close prices.
    volume (pd.Series): Series of volumes.

    Returns:
    amihud (pd.Series): Amihud Illiquidity values.
    """
    daily_return = close.pct_change()
    amihud = daily_return.abs() / volume
    return amihud

In [11]:
def fetch_data(data):
    stock_data=data.copy()
    High=stock_data['High']
    Low=stock_data['Low']
    Open=stock_data['Open']
    Close=stock_data['Close']
    Volume=stock_data['Volume']
    High5=stock_data['High_5min']
    Low5=stock_data['Low_5min']
    Close5=stock_data['Close_5min']
    Volume5=stock_data['Volume_5min']
    High15=stock_data['High_15min']
    Low15=stock_data['Low_15min']
    Close15=stock_data['Close_15min']
    Volume15=stock_data['Volume_15min']
    High30=stock_data['High_30min']
    Low30=stock_data['Low_30min']
    Close30=stock_data['Close_30min']
    Volume30=stock_data['Volume_30min']
    stock_data['price']=(stock_data['Open']+stock_data['Close']+stock_data['High']+stock_data['Low'])/4
    
    stock_data['RSI'] = talib.RSI(Close, timeperiod=14)
    
    # 佳庆指标（Chaikin Oscillator），该指标基于AD曲线的指数移动均线而计算得到。属于趋势型因子
    stock_data['ADOSC'] = talib.ADOSC(High, Low, Close, Volume, fastperiod=3, slowperiod=10)
    # 平均动向指数，DMI因子的构成部分。属于趋势型因子
    stock_data['ADX'] = talib.ADX(High, Low, Close,timeperiod=14)
    # 绝对价格振荡指数
    stock_data['APO'] = talib.APO(Close, fastperiod=12, slowperiod=26)
    # 均势指标
    stock_data['BOP'] = talib.BOP(Open, High, Low, Close)
    stock_data['SAR'] = talib.SAR(High, Low, acceleration=0.02, maximum=0.2)
    # 威廉指标
    stock_data['WILLR'] = talib.WILLR(High, Low, Close, timeperiod=14)
    # PLUS_DI 更向指示器
    stock_data['PLUS_DM'] = talib.PLUS_DM(High, Low, timeperiod=14)

    stock_data['SLOWK_30min'],stock_data['SLOWD_30min']=talib.STOCH(High30,Low5,Close30)
    stock_data['amihud']=calculate_amihud_illiquidity(stock_data['Close'],stock_data['Volume'])
    stock_data['amihud_30min']=calculate_amihud_illiquidity(stock_data['Close_30min'],stock_data['Volume_30min'])
    stock_data['VROCP6_5min'] = talib.ROCP(Volume5, timeperiod=6)
    stock_data['VROCP6_15min'] = talib.ROCP(Volume15, timeperiod=6)

    # Add code to keep only the specified columns
    cols_to_keep = ['time','price','Volume', 'amihud', 'SLOWK_30min', 'VROCP6_5min', 'VROCP6_15min', 'amihud_30min','PLUS_DM', 'WILLR', 'RSI', 'APO', 'BOP', 'ADX', 'ADOSC', 'SAR']
    stock_data = stock_data[cols_to_keep]

    stock_data[np.isinf(stock_data)] = np.nan
    #stock_data.dropna(inplace=True)
    stock_data = stock_data.fillna(method='bfill')
    stock_data['time'] = pd.to_datetime(stock_data['time'])
    stock_data.set_index('time', inplace=True)

    return stock_data

In [5]:
class CryptoTradingEnv(gym.Env):
    metadata = {'render.modes': ['human', 'system', 'none']}
    def __init__(self, df, lookback_window_size=10, initial_balance=25000, commission=0.0005, serial=True,render_mode='none'):
        super(CryptoTradingEnv, self).__init__()
        self.df = df.dropna()
        self.render_mode = render_mode
        self.lookback_window_size = lookback_window_size
        self.initial_balance = initial_balance
        self.commission = commission
        self.serial = serial
        self.action_space = spaces.MultiDiscrete([3, 10])
        self.observation_space = spaces.Box(low=0, high=1, shape=(20, lookback_window_size + 1), dtype=np.float16)
        self.scaler = MinMaxScaler()
        self.balance_list=[self.initial_balance]
        
    def _get_current_price(self):
        #Open=self.df['Open'].values[self.frame_start + self.current_step]
        #Close=self.df['Close'].values[self.frame_start + self.current_step]
        #High=self.df['High'].values[self.frame_start + self.current_step]
        #Low=self.df['Low'].values[self.frame_start + self.current_step]
        price=self.df['price'].values[self.frame_start + self.current_step]
        return price
    
    def get_max_volume(self):
        return 0.25*self.df['Volume'].values[self.frame_start + self.current_step]
    
    def _next_observation(self):
        end = self.current_step + self.lookback_window_size + 1
        scaled_df = self.active_df.values[:end].astype('float64')
        scaled_df = self.scaler.fit_transform(scaled_df)
        scaled_df = pd.DataFrame(scaled_df, columns=self.df.columns)

        cols_to_keep = ['price', 'Volume', 'amihud', 'SLOWK_30min', 'VROCP6_5min', 'VROCP6_15min', 'amihud_30min',
                        'PLUS_DM', 'WILLR', 'RSI', 'APO', 'BOP', 'ADX', 'ADOSC', 'SAR']

        obs = np.array([scaled_df[col].values[self.current_step:end] for col in cols_to_keep])


        scaled_history = self.scaler.fit_transform(self.account_history)

        obs = np.append(
            obs, scaled_history[:, -(self.lookback_window_size + 1):], axis=0)

        return obs

    def _reset_session(self):
        self.current_step = 0
        self.reward_len=0

        if self.serial:
            self.steps_left = len(self.df) - self.lookback_window_size - 1
            self.frame_start = self.lookback_window_size
        else:
            self.steps_left = np.random.randint(1, MAX_TRADING_SESSION)
            self.frame_start = np.random.randint(
                self.lookback_window_size, len(self.df) - self.steps_left)

        self.active_df = self.df[self.frame_start - self.lookback_window_size:
                                 self.frame_start + self.steps_left]
        
    def reset(self, **kwargs):
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.btc_held = 0
        self._reset_session()

        self.account_history = np.repeat([
            [self.balance],
            [0],
            [0],
            [0],
            [0]
        ], self.lookback_window_size + 1, axis=1)
        self.trades = []

        return self._next_observation()

    def _take_action(self, action, current_price):
        action_type = action[0]
        amount = action[1] / 10
        btc_bought = 0
        btc_sold = 0
        cost = 0
        sales = 0
        max_volume=self.get_max_volume()
        #print('last value: ',self.net_worth)
        #print('last hold: ',self.btc_held)
        #print('last cash: ', self.balance)
        if action_type < 1:
            btc_bought = min((self.balance / current_price) * amount,max_volume)
            cost = btc_bought * current_price * (1 + self.commission)

            self.btc_held += btc_bought
            self.balance -= cost

        elif action_type < 2:
            btc_sold = min(self.btc_held * amount,max_volume)
            sales = btc_sold * current_price * (1 - self.commission)

            self.btc_held -= btc_sold
            self.balance += sales

        if btc_sold > 0 or btc_bought > 0:
            self.trades.append({'step': self.frame_start + self.current_step,
                                'amount': btc_sold if btc_sold > 0 else btc_bought, 'total': sales if btc_sold > 0 else cost,
                                'type': "sell" if btc_sold > 0 else "buy"})

        self.net_worth = self.balance + self.btc_held * current_price
        self.balance_list.append(self.net_worth)
        self.account_history = np.append(self.account_history, [
            [self.balance],
            [btc_bought],
            [cost],
            [btc_sold],
            [sales]
        ], axis=1)
        #print('current price: ',current_price)
        #print('current value: ',self.net_worth)
        #print('current hold: ',self.btc_held)
        #print('current cash: ', self.balance)
        #print('Balance: ', self.balance)
        #print('Bought Shares: ' ,btc_bought)
        #print('Bought Cost: ', cost)
        #print('Sold Shares: ', btc_sold)
        #print('Sold Sales: ',sales)


    def step(self, action):
        #print([self.current_step,self.steps_left,self.reward_len])
        current_price = self._get_current_price()

        prev_net_worth = self.net_worth

        self._take_action(action, current_price)

        self.steps_left -= 1
        self.current_step += 1

        if self.steps_left == 0:
        #    self.balance += self.btc_held * current_price
        #    self.btc_held = 0

            self._reset_session()

        obs = self._next_observation()
        length=min(11,self.current_step)
        balances = np.array(self.balance_list)[-length:]
        returns=np.diff(balances)/balances[:-1]
        #sharpe = sharpe_ratio(returns)
        sortino = sortino_ratio(returns)
        if np.isnan(sortino):
            sortino=0
        if sortino==np.inf:
            sortino=10
        reward = sortino
        self.reward_len+=1
        
        done = self.net_worth <= 0
        
        return obs, reward, done, {}
    
    def render(self, mode='human', **kwargs):
        if mode == 'system':
            print('Price: ' + str(self._get_current_price()))
            print(
                'Bought: ' + str(self.account_history[2][self.current_step + self.frame_start]))
            print(
                'Sold: ' + str(self.account_history[4][self.current_step + self.frame_start]))
            print('Net worth: ' + str(self.net_worth))

In [6]:
def optimize_agent(trial):

    data=training_data.copy()
    train_size =len(data)-len(testing_data)  # 80% for training, 20% for evaluation
    train = data[:train_size]
    evaluate = data[train_size:]

    # Create the training environment
    train_env = CryptoTradingEnv(train,initial_balance=50477.5711, commission=0.0005, render_mode='system')

    # Define hyperparameters using the trial object
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    n_steps = trial.suggest_categorical("n_steps", [128, 256, 512, 1024, 2048])
    gamma = trial.suggest_float("gamma", 0.9, 0.9999, log=True)
    gae_lambda = trial.suggest_float("gae_lambda", 0.8, 0.99)
    clip_range = trial.suggest_float("clip_range", 0.1, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])

    # Setup the model
    model = RecurrentPPO("MlpLstmPolicy", train_env, verbose=0, 
                learning_rate=learning_rate,
                n_steps=n_steps,
                gamma=gamma,
                gae_lambda=gae_lambda,
                clip_range=clip_range,
                batch_size=batch_size,
                tensorboard_log="./tensorboard/",
                device='cuda')
    
    # Train the model
    model.learn(total_timesteps=10000)  # Train on 80% of the training data
    eval_env = CryptoTradingEnv(evaluate,initial_balance=50477.5711, commission=0.0005,render_mode='system')
    obs = eval_env.reset()
    rewards_list,done=[],False
    for _ in range(len(evaluate)):
        action, _states = model.predict(obs)
        obs, rewards, done, info = eval_env.step(action)
        rewards_list.append(rewards)

    return np.mean(rewards_list)



In [105]:
btc_data=pd.read_csv('csv\\BTCUSD_1m_btc_usd_60.csv')
resample_btc=resample_data(btc_data)
btc_feature=fetch_data(resample_btc)
# Define your date ranges
start_date = '2021-09-14 00:00:00'
end_date = '2023-04-19 00:00:00'

# Split the data into training and testing sets
#training_data = btc_feature[start_date:end_date]
testing_data_btc = btc_feature[end_date:][1:]

eth_data=pd.read_csv('csv\\ETHUSD_1m_eth_usd_60.csv')
resample_eth=resample_data(eth_data)
eth_feature=fetch_data(resample_eth)

# Split the data into training and testing sets
#training_data = eth_feature[start_date:end_date]
testing_data_eth = eth_feature[end_date:][1:]

ltc_data=pd.read_csv('csv\\LTCUSD_1m_ltc_usd_60.csv')
resample_ltc=resample_data(ltc_data)
ltc_feature=fetch_data(resample_ltc)

# Split the data into training and testing sets
#training_data = ltc_feature[start_date:end_date]
testing_data_ltc = ltc_feature[end_date:][1:]

xrp_data = pd.read_csv('csv\\XRPUSD_1m_xrp_usd_60.csv')
resample_xrp=resample_data(xrp_data)
xrp_feature=fetch_data(resample_xrp)

# Split the data into training and testing sets
#training_data = xrp_feature[start_date:end_date]
testing_data_xrp = xrp_feature[end_date:][1:]

In [46]:
loaded_model_btc = RecurrentPPO.load("RecurrentPPO_BTC")
loaded_model_eth = RecurrentPPO.load("RecurrentPPO_ETH")
loaded_model_ltc = RecurrentPPO.load("RecurrentPPO_LTC")
loaded_model_xrp = RecurrentPPO.load("RecurrentPPO_XRP")

In [67]:
backtestdata=pd.read_csv('Min Bar Balance.csv')

In [65]:
backtestdata

,Date,Net Worth ETH,Action ETH,Net Worth LTC,Action LTC,Net Worth XRP,Action XRP,Net Worth BTC,Action BTC,Total Worth
0,2023-04-19 00:01:00,33632.030100,2.0,14483.748903,0.0,1406.647599,0.0,50477.571100,1.0,99999.997702
1,2023-04-19 00:02:00,33632.030100,2.0,14483.748903,0.0,1406.601722,0.0,50477.547998,0.0,99999.928724
2,2023-04-19 00:03:00,33632.030100,2.0,14483.748903,0.0,1406.506328,0.0,50477.541341,1.0,99999.826672
3,2023-04-19 00:04:00,33632.030100,2.0,14483.748903,0.0,1406.180869,0.0,50477.549336,0.0,99999.509208
4,2023-04-19 00:05:00,33632.030100,2.0,14483.748903,0.0,1406.438660,0.0,50477.556614,0.0,99999.774277
...,...,...,...,...,...,...,...,...,...,...
19141,2023-05-02 23:55:00,33825.144018,2.0,15314.527268,0.0,1375.201421,0.0,54320.361213,2.0,104835.233920
19142,2023-05-02 23:56:00,33825.073175,2.0,15320.663184,0.0,1374.217841,0.0,54333.508181,0.0,104853.462381
19143,2023-05-02 23:57:00,33823.904266,2.0,15304.115565,1.0,1374.942593,2.0,54333.508181,0.0,104836.470605
19144,2023-05-02 23:58:00,33823.904266,2.0,15304.115565,1.0,1373.234234,0.0,54335.010945,2.0,104836.265010


In [73]:
btc_data.rename(columns={'time': 'Date'}, inplace=True)
eth_data.rename(columns={'time': 'Date'}, inplace=True)
ltc_data.rename(columns={'time': 'Date'}, inplace=True)
xrp_data.rename(columns={'time': 'Date'}, inplace=True)

# Function to calculate the mean price
def calculate_mean_price(df):
    return df[['Open', 'High', 'Low', 'Close']].mean(axis=1)

# Merge the dataframes on Date
merged_btc = pd.merge(backtestdata, btc_data, on='Date', how='left')
merged_eth = pd.merge(backtestdata, eth_data, on='Date', how='left')
merged_ltc = pd.merge(backtestdata, ltc_data, on='Date', how='left')
merged_xrp = pd.merge(backtestdata, xrp_data, on='Date', how='left')

# Calculate mean prices
merged_btc['Mean Price BTC'] = calculate_mean_price(merged_btc)
merged_eth['Mean Price ETH'] = calculate_mean_price(merged_eth)
merged_ltc['Mean Price LTC'] = calculate_mean_price(merged_ltc)
merged_xrp['Mean Price XRP'] = calculate_mean_price(merged_xrp)

# Calculate the quantity of each cryptocurrency owned
backtestdata['BTC Quantity'] = merged_btc['Net Worth BTC'] / merged_btc['Mean Price BTC']
backtestdata['ETH Quantity'] = merged_eth['Net Worth ETH'] / merged_eth['Mean Price ETH']
backtestdata['LTC Quantity'] = merged_ltc['Net Worth LTC'] / merged_ltc['Mean Price LTC']
backtestdata['XRP Quantity'] = merged_xrp['Net Worth XRP'] / merged_xrp['Mean Price XRP']

# Calculate quantity change for each asset
backtestdata['BTC Quantity Change'] = backtestdata['BTC Quantity'].diff()
backtestdata['ETH Quantity Change'] = backtestdata['ETH Quantity'].diff()
backtestdata['LTC Quantity Change'] = backtestdata['LTC Quantity'].diff()
backtestdata['XRP Quantity Change'] = backtestdata['XRP Quantity'].diff()

In [74]:
backtestdata

,Date,Net Worth ETH,Action ETH,Net Worth LTC,Action LTC,Net Worth XRP,Action XRP,Net Worth BTC,Action BTC,Total Worth,BTC Quantity,ETH Quantity,LTC Quantity,XRP Quantity,BTC Quantity Change,ETH Quantity Change,LTC Quantity Change,XRP Quantity Change
0,2023-04-19 00:01:00,33632.030100,2.0,14483.748903,0.0,1406.647599,0.0,50477.571100,1.0,99999.997702,1.663072,15.999634,142.426213,2639.262245,NaN,NaN,NaN,NaN
1,2023-04-19 00:02:00,33632.030100,2.0,14483.748903,0.0,1406.601722,0.0,50477.547998,0.0,99999.928724,1.663263,16.012202,142.448626,2637.073318,0.000191,0.012569,0.022412,-2.188927
2,2023-04-19 00:03:00,33632.030100,2.0,14483.748903,0.0,1406.506328,0.0,50477.541341,1.0,99999.826672,1.664360,16.032812,142.448626,2631.701576,0.001097,0.020610,0.000000,-5.371742
3,2023-04-19 00:04:00,33632.030100,2.0,14483.748903,0.0,1406.180869,0.0,50477.549336,0.0,99999.509208,1.663195,16.028228,142.448626,2625.529088,-0.001165,-0.004585,0.000000,-6.172487
4,2023-04-19 00:05:00,33632.030100,2.0,14483.748903,0.0,1406.438660,0.0,50477.556614,0.0,99999.774277,1.663360,16.019066,142.448626,2621.556152,0.000165,-0.009161,0.000000,-3.972937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19141,2023-05-02 23:55:00,33825.144018,2.0,15314.527268,0.0,1375.201421,0.0,54320.361213,2.0,104835.233920,1.896727,18.090247,173.421743,2959.395341,0.001731,-0.015834,0.000000,-4.010249
19142,2023-05-02 23:56:00,33825.073175,2.0,15320.663184,0.0,1374.217841,0.0,54333.508181,0.0,104853.462381,1.896954,18.111519,173.538390,2957.246884,0.000227,0.021272,0.116647,-2.148457
19143,2023-05-02 23:57:00,33823.904266,2.0,15304.115565,1.0,1374.942593,2.0,54333.508181,0.0,104836.470605,1.895531,18.101201,173.563278,2959.427446,-0.001423,-0.010318,0.024888,2.180562
19144,2023-05-02 23:58:00,33823.904266,2.0,15304.115565,1.0,1373.234234,0.0,54335.010945,2.0,104836.265010,1.895881,18.101201,173.748502,2955.018930,0.000350,0.000000,0.185225,-4.408516


In [88]:
function_backtest = pd.DataFrame(index=backtestdata.index)

In [89]:
function_backtest['time'] = backtestdata['Date']
function_backtest['BTC Quantity Change'] = backtestdata['BTC Quantity Change']
function_backtest['ETH Quantity Change'] = backtestdata['ETH Quantity Change']
function_backtest['LTC Quantity Change'] = backtestdata['LTC Quantity Change']
function_backtest['XRP Quantity Change'] = backtestdata['XRP Quantity Change'] 
function_backtest=function_backtest.fillna(0)

In [90]:
function_backtest

,time,BTC Quantity Change,ETH Quantity Change,LTC Quantity Change,XRP Quantity Change
0,2023-04-19 00:01:00,0.000000,0.000000,0.000000,0.000000
1,2023-04-19 00:02:00,0.000191,0.012569,0.022412,-2.188927
2,2023-04-19 00:03:00,0.001097,0.020610,0.000000,-5.371742
3,2023-04-19 00:04:00,-0.001165,-0.004585,0.000000,-6.172487
4,2023-04-19 00:05:00,0.000165,-0.009161,0.000000,-3.972937
...,...,...,...,...,...
19141,2023-05-02 23:55:00,0.001731,-0.015834,0.000000,-4.010249
19142,2023-05-02 23:56:00,0.000227,0.021272,0.116647,-2.148457
19143,2023-05-02 23:57:00,-0.001423,-0.010318,0.024888,2.180562
19144,2023-05-02 23:58:00,0.000350,0.000000,0.185225,-4.408516


In [91]:
# Columns representing the quantity changes for each cryptocurrency
columns = ['BTC Quantity Change', 'ETH Quantity Change', 'LTC Quantity Change', 'XRP Quantity Change']

# Calculate the cumulative quantity for each cryptocurrency
for column in columns:
    function_backtest[column + ' Cumulative'] = function_backtest[column].cumsum()

In [92]:
function_backtest

,time,BTC Quantity Change,ETH Quantity Change,LTC Quantity Change,XRP Quantity Change,BTC Quantity Change Cumulative,ETH Quantity Change Cumulative,LTC Quantity Change Cumulative,XRP Quantity Change Cumulative
0,2023-04-19 00:01:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2023-04-19 00:02:00,0.000191,0.012569,0.022412,-2.188927,0.000191,0.012569,0.022412,-2.188927
2,2023-04-19 00:03:00,0.001097,0.020610,0.000000,-5.371742,0.001288,0.033178,0.022412,-7.560669
3,2023-04-19 00:04:00,-0.001165,-0.004585,0.000000,-6.172487,0.000123,0.028594,0.022412,-13.733157
4,2023-04-19 00:05:00,0.000165,-0.009161,0.000000,-3.972937,0.000287,0.019433,0.022412,-17.706093
...,...,...,...,...,...,...,...,...,...
19141,2023-05-02 23:55:00,0.001731,-0.015834,0.000000,-4.010249,0.233655,2.090613,30.995530,320.133097
19142,2023-05-02 23:56:00,0.000227,0.021272,0.116647,-2.148457,0.233882,2.111885,31.112176,317.984639
19143,2023-05-02 23:57:00,-0.001423,-0.010318,0.024888,2.180562,0.232459,2.101567,31.137064,320.165201
19144,2023-05-02 23:58:00,0.000350,0.000000,0.185225,-4.408516,0.232809,2.101567,31.322289,315.756685


In [102]:
# Define the columns for cumulative quantities
quantity_change_columns = ['BTC Quantity Change', 'ETH Quantity Change', 'LTC Quantity Change', 'XRP Quantity Change']
cumulative_columns = [col + ' Cumulative' for col in quantity_change_columns]

# Calculate the cumulative sums
for col in quantity_change_columns:
    function_backtest[col + ' Cumulative'] = function_backtest[col].cumsum()

# Create the new DataFrame 'position' with cumulative quantities and the 'time' column
position_columns = ['time'] + cumulative_columns
position_data = function_backtest[position_columns]

# Rename columns in 'position' DataFrame for clarity, except for 'time'
new_column_names = ['time'] + ['BTC Position', 'ETH Position', 'LTC Position', 'XRP Position']
position_data .columns = new_column_names

# Set the 'time' column as the index of the position DataFrame
position_data .set_index('time', inplace=True)

In [103]:
position_data 

,BTC Position,ETH Position,LTC Position,XRP Position
time,,,,
2023-04-19 00:01:00,0.000000,0.000000,0.000000,0.000000
2023-04-19 00:02:00,0.000191,0.012569,0.022412,-2.188927
2023-04-19 00:03:00,0.001288,0.033178,0.022412,-7.560669
2023-04-19 00:04:00,0.000123,0.028594,0.022412,-13.733157
2023-04-19 00:05:00,0.000287,0.019433,0.022412,-17.706093
...,...,...,...,...
2023-05-02 23:55:00,0.233655,2.090613,30.995530,320.133097
2023-05-02 23:56:00,0.233882,2.111885,31.112176,317.984639
2023-05-02 23:57:00,0.232459,2.101567,31.137064,320.165201


In [ ]:
def handle_bar(counter,  # a counter for number of minute bars that have already been tested
               time,  # current time in string format such as "2018-07-30 00:30:00"
               data,  # data for current minute bar (in format 2)
               init_cash,  # your initial cash, a constant
               transaction,  # transaction ratio, a constant
               cash_balance,  # your cash balance at current minute
               crypto_balance,  # your crpyto currency balance at current minute
               total_balance,  # your total balance at current minute
               position_current,  # your position for 4 crypto currencies at this minute
               memory  # a class, containing the information you saved so far
               ):
    # Here you should explain the idea of your strategy briefly in the form of Python comment.
    # You can also attach facility files such as text & image & table in your team folder to illustrate your idea

    """
    We use OpenAI's gym to configure a simulated trading environment, in the built-in environment to consider the 
    transaction costs, maximum trading volume, cash balance, asset balance and other variables already considered by this function, 
    so in order to meet the needs of this function, we use the csv file of our own test results to adapt to the desired content, 
    so there is NO guarantee that there will NOT be some error. 
    If you need details, you can access our detailed policy file 
    and use the trained model to test back directly instead of using this function.
    """
    position_list=position_data.loc[time].tolist(0)
    
    if (counter == 0):
        position=[0,0,0,0]
    else:
        position = position_current  # load current position
        position=position_list 
    return position, memory